In [7]:
import pandas as pd

from backtesting import Strategy, Backtest
from backtesting.lib import crossover

import talib
import pandas_ta

import warnings
import strategies

warnings.filterwarnings("ignore")

In [8]:
ticker = "SBIN"
df = pd.read_parquet(f"../res/data/{ticker}.parquet")
df

,Open,High,Low,Close,Volume
ts,,,,,
2019-02-04 03:45:00,282.40,282.45,279.55,280.85,991844
2019-02-04 03:50:00,280.85,284.30,280.10,282.80,798747
2019-02-04 03:55:00,282.85,283.00,281.30,281.95,372364
2019-02-04 04:00:00,281.95,282.00,281.20,281.50,324310
2019-02-04 04:05:00,281.35,281.80,280.00,280.00,474058
...,...,...,...,...,...
2024-01-25 09:35:00,610.40,612.70,610.40,612.65,1463474
2024-01-25 09:40:00,612.70,613.30,611.95,612.85,1205205
2024-01-25 09:45:00,612.90,613.50,612.55,613.00,1436920


In [9]:
class SupertrendStrategy(Strategy):
    length = 10

    def supertrend(self, df, length):
        supertrend = pandas_ta.supertrend(df.High, df.Low, df.Close, length=length)
        return supertrend.to_numpy().T[1]

    def init(self):
        df = self.data.df
        self.dir = self.I(self.supertrend, df, self.length)

    def next(self):
        if self.dir[-1] * self.dir[-2] > 0:
            return

        if self.dir[-1] > 0:
            self.buy()
        elif self.dir[-1] < 0:
            self.sell()

## Optimization


### Training


In [10]:
bt = Backtest(
    df.iloc[: len(df) // 2],
    SupertrendStrategy,
    cash=50_000,
    commission=20 / 50_000,
    exclusive_orders=True,
)
stats = bt.optimize(length=[x for x in range(5, 32, 2)], maximize="Sharpe Ratio")
print("Strategy:", stats._strategy)
print("Sharpe ratio:", stats["Sharpe Ratio"])

bt.plot()

Backtest.optimize:   0%|          | 0/14 [00:00<?, ?it/s]

Strategy: SupertrendStrategy(length=7)
Sharpe ratio: 1.034073416094872


GridPlot(id='p2500', ...)

### Testing


In [11]:
bt = Backtest(
    df.iloc[len(df) // 2 :],
    SupertrendStrategy,
    cash=50_000,
    commission=20 / 50_000,
    exclusive_orders=True,
)
print(bt.run(length=stats._strategy.length))
bt.plot()

Start                     2021-08-02 09:45:00
End                       2024-01-25 09:55:00
Duration                    906 days 00:10:00
Exposure Time [%]                   99.824287
Equity Final [$]                  58652.40728
Equity Peak [$]                    72260.6146
Return [%]                          17.304815
Buy & Hold Return [%]               41.428079
Return (Ann.) [%]                    6.724643
Volatility (Ann.) [%]               25.181401
Sharpe Ratio                         0.267048
Sortino Ratio                        0.430199
Calmar Ratio                         0.304839
Max. Drawdown [%]                  -22.059673
Avg. Drawdown [%]                   -1.775212
Max. Drawdown Duration      260 days 05:25:00
Avg. Drawdown Duration       10 days 15:02:00
# Trades                                 1062
Win Rate [%]                        35.216573
Best Trade [%]                       8.328546
Worst Trade [%]                     -3.074608
Avg. Trade [%]                    

GridPlot(id='p2883', ...)

## Comparison of Strategies


In [12]:
strats = [
    strategies.SupertrendStrategy,
    strategies.BBandReversion,
    strategies.MACDCross,
    strategies.EMACross,
    strategies.VWAPCross,
]


def backtest_strategies(df, strategies):
    res = []
    colname = []
    for strat in strategies:
        bt = Backtest(df, strat, cash=10_000, exclusive_orders=True)
        strat_name = bt._strategy.__name__
        res.append(bt.run().to_frame())
        colname.append(strat_name)
        bt.plot(
            filename="../res/backtests/" + strat_name + ".html",
            # resample="5D",
            smooth_equity=True,
            open_browser=False,
        )
    res = pd.concat(res, axis=1)
    res.columns = colname
    return res


backtest_strategies(df, strats)

,SupertrendStrategy,BBandReversion,MACDCross,EMACross,VWAPCross
Start,2019-02-04 03:45:00,2019-02-04 03:45:00,2019-02-04 03:45:00,2019-02-04 03:45:00,2019-02-04 03:45:00
End,2024-01-25 09:55:00,2024-01-25 09:55:00,2024-01-25 09:55:00,2024-01-25 09:55:00,2024-01-25 09:55:00
Duration,1816 days 06:10:00,1816 days 06:10:00,1816 days 06:10:00,1816 days 06:10:00,1816 days 06:10:00
Exposure Time [%],99.950106,99.928413,99.962037,99.931667,95.447742
Equity Final [$],87192.05,34529.65,13889.95,105796.95,599266.141
Equity Peak [$],97928.65,37603.45,23146.7,113621.45,606446.191
Return [%],771.9205,245.2965,38.8995,957.9695,5892.66141
Buy & Hold Return [%],118.978102,118.978102,118.978102,118.978102,118.978102
Return (Ann.) [%],55.762513,28.843254,7.260329,61.811788,130.740292
Volatility (Ann.) [%],53.735484,42.771784,35.606475,51.869493,66.498552


# Ensemble Strategies


In [17]:
class SuperTrendVWAP(Strategy):
    length = 10
    ema_window = 16

    def vwapFunc(self, df):
        vwap = pandas_ta.vwap(df.High, df.Low, df.Close, df.Volume)
        return vwap.to_numpy().T

    def supertrend(self, df, length):
        supertrend = pandas_ta.supertrend(df.High, df.Low, df.Close, length=length)
        return supertrend.to_numpy().T[1]

    def init(self):
        df = self.data.df
        self.dir = self.I(self.supertrend, df, self.length)
        self.vwap = self.I(self.vwapFunc, df)
        self.ema = self.I(talib.EMA, df.Open, self.ema_window)

    def next(self):
        if self.dir > 0 and crossover(self.ema, self.vwap):
            self.buy()
        elif self.dir < 0 and crossover(self.vwap, self.ema):
            self.sell()

In [18]:
bt = Backtest(
    df, SuperTrendVWAP, cash=50_000, commission=20 / 50_000, exclusive_orders=True
)
bt.run()

Start                     2019-02-04 03:45:00
End                       2024-01-25 09:55:00
Duration                   1816 days 06:10:00
Exposure Time [%]                   99.946852
Equity Final [$]                 178939.44842
Equity Peak [$]                   238124.8156
Return [%]                         257.878897
Buy & Hold Return [%]              118.978102
Return (Ann.) [%]                   30.090186
Volatility (Ann.) [%]               43.431516
Sharpe Ratio                         0.692819
Sortino Ratio                        1.464911
Calmar Ratio                          0.84837
Max. Drawdown [%]                  -35.468217
Avg. Drawdown [%]                   -1.495075
Max. Drawdown Duration     1056 days 01:20:00
Avg. Drawdown Duration        5 days 10:42:00
# Trades                                 1746
Win Rate [%]                        45.761741
Best Trade [%]                      18.355025
Worst Trade [%]                    -12.141597
Avg. Trade [%]                    

In [19]:
bt.plot()

GridPlot(id='p5280', ...)